## **Parte Teórica**

# 1. Bucles de retroalimentación

## 1.a Crecimiento exponencial (bucle de refuerzo)
Un **bucle de refuerzo** es aquel en el que la salida del sistema se realimenta de forma positiva, amplificando el cambio inicial.  
- **Por ejemplo** población $P$ con tasa de crecimiento proporcional al tamaño actual.  
  - Flujo: $\text{nacimientos} = r\,P$  
  - Ecuación diferencial:  
    $$\frac{dP}{dt} = r\,P$$  
  - Cada nueva unidad de población genera más nacimientos, acelerando el crecimiento de forma **exponencial**.

## 1.b Equilibrio (bucle de equilibrio)
Un **bucle de equilibrio** introduce una realimentación negativa: la salida actúa contrarrestando el cambio inicial para acercar el sistema a un valor de referencia o meta.  
- **Por ejemplo:** termostato controlando la temperatura $T$.  
  - Stock: temperatura ambiente $T$  
  - Meta: temperatura objetivo $T_{\text{ref}}$  
  - Flujo de calefacción:  
    $$Q = k\,(T_{\text{ref}} - T)$$  
  - Cuando $T < T_{\text{ref}}$, $Q>0$ calienta; si $T > T_{\text{ref}}$, $Q<0$ enfría.  
  - El bucle reduce continuamente la diferencia $(T_{\text{ref}} - T)$, manteniendo el **equilibrio**.

---

# 2. Modelo epidémico SIR

El modelo SIR divide a la población en tres **stocks** y describe los **flujos** entre ellos.

## 2.a Etiquetado de stocks y flujos

- **Stocks**  
  - **S**: Susceptibles (personas que aún pueden infectarse)  
  - **I**: Infectados (personas actualmente contagiadas y transmisoras)  
  - **R**: Recuperados (o removidos; ya no participan en la transmisión)

- **Flujos**  
  1. **Infección**  
     $$\text{flujo}_{S\to I} = \beta\,S\,I$$  
     Tasa a la que susceptibles se infectan al contacto con infectados.  
  2. **Recuperación**  
     $$\text{flujo}_{I\to R} = \gamma\,I$$  
     Tasa a la que los infectados dejan de ser transmisores (recuperados o fallecidos).

## 2.b Bucles de retroalimentación que rigen la infección

1. **Bucle reforzador de infección**  
   - **Camino:**  
     $I \xrightarrow{\beta S I} I$  
   - Más infectados $\Rightarrow$ más contactos $\Rightarrow$ más nuevas infecciones $\Rightarrow$ aún más infectados.

2. **Bucle equilibrante de recuperación**  
   - **Camino:**  
     $I \xrightarrow{\gamma I} R$  
   - A medida que crece $I$, aumenta el flujo de recuperación, reduciendo la cantidad de infectados y frenando la epidemia.

---

# 3. Por qué los sistemas no lineales requieren simulación numérica

Cuando un flujo depende de productos de variables (por ejemplo, $\beta\,S\,I$) el sistema de ecuaciones diferenciales resultante es **no lineal**. En la práctica:

1. **Acoplamiento entre variables**  
   - El término $\beta\,S\,I$ mezcla dos stocks, de modo que la tasa de cambio de uno depende del valor instantáneo del otro.

2. **Imposibilidad de solución analítica sencilla**  
   - Salvo casos muy simplificados, no existe una fórmula cerrada para $S(t), I(t), R(t)$.  
   - La no linealidad impide separar variables en integrales elementales.

3. **Necesidad de métodos numéricos**  
   - Se usan algoritmos de integración paso a paso (por ejemplo, **Euler**, **Runge–Kutta**).  
   - Estos métodos aproximan la evolución en pequeños intervalos $\Delta t$, actualizando los valores de $S$, $I$ y $R$ iterativamente.

 
